In [1]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import subprocess
from docx.shared import Pt
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
import glob
from zipfile import ZipFile 


In [2]:
class MailMerge():
    def __init__(self, dataset ,label):
        self.dataset = dataset
        self.label = label

    def row_to_pdf(self, data):
        for i in data.keys():
            if str(data.get(i)) == 'nan':
                data[i] = ''
                
        f = open('template.docx', 'rb')
        doc = Document(f)
        
        ## General Information
        doc.tables[0].cell(0, 1).text = data['doctor_name']
        doc.tables[0].cell(1, 1).text = data['doctor_department']
        doc.tables[0].cell(2, 1).text = data['doctor_sip']
        doc.tables[0].cell(3, 1).text = data['doctor_str']
        doc.tables[0].cell(0, 3).text = data['name']
        doc.tables[0].cell(1, 3).text = data['gender']
        doc.tables[0].cell(2, 3).text = data['dob']
        doc.tables[0].cell(3, 3).text = str(data['card_no']).replace('.0', '')
        doc.tables[0].cell(4, 3).text = data['payor']
        doc.tables[0].cell(5, 3).text = data['corporate']
        
        for i in range(6):
            if i < 4:
                doc.tables[0].cell(i, 1).paragraphs[0].runs[0].font.size = Pt(8)
            doc.tables[0].cell(i, 3).paragraphs[0].runs[0].font.size = Pt(8)
        
        ## complaint & diagnosis
        doc.paragraphs[2].text = data['chief_complaints']
        doc.paragraphs[2].runs[0].font.size= Pt(8)
        doc.paragraphs[4].text = str(data['diagnosis']) if str(data['diagnosis']) != '' else '' + (', ' + str(data['suggestion'])) if str(data['suggestion']) != '' else '' 
        doc.paragraphs[4].runs[0].font.size= Pt(8)
        
        doc.paragraphs[6].text = data['icdx']
        doc.paragraphs[6].runs[0].font.size= Pt(8)
        
        ## Consultation Detail
        doc.tables[1].cell(1, 1).text = str(data['consult_id']).replace('.0', '')
        doc.tables[1].cell(1, 2).text = str(data['claim_id']).replace('.0', '')
        doc.tables[1].cell(1, 3).text = data['date'].strftime('%d/%m/%Y') + ' ' + data['time']
        doc.tables[1].cell(1, 4).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
        doc.tables[1].cell(1, 7).text = 'IDR '+ str(data['consult_fee']).replace('.0', '')
        doc.tables[1].cell(2, 1).text = str(data['order_id']).replace('.0', '')
        
        
        doc.tables[1].cell(1, 1).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 2).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 3).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 4).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(1, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(2, 1).paragraphs[0].runs[0].font.size= Pt(8)
        
        for i in range(1,13):
            j = i + 2
            if str(data[f'obat_{i}']) != '':
                doc.tables[1].cell(j, 0).text = data[f'obat_{i}']
                doc.tables[1].cell(j, 0).paragraphs[0].alignment = 2
                doc.tables[1].cell(j, 4).text = 'IDR '+ str(data[f'total_{i}']).replace('.0', '')
                doc.tables[1].cell(j, 5).text = str(data[f'jumlah_{i}']).replace('.0', '')
                doc.tables[1].cell(j, 6).text = data[f'unit_obat_{i}']
                doc.tables[1].cell(j, 7).text = 'IDR '+ str(data[f'total_{i}']).replace('.0', '')
        
                doc.tables[1].cell(j, 6).paragraphs[0].runs[0].italic = True
                
                doc.tables[1].cell(j, 0).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 4).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 5).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 6).paragraphs[0].runs[0].font.size= Pt(8)
                doc.tables[1].cell(j, 7).paragraphs[0].runs[0].font.size= Pt(8)
        
        doc.tables[1].cell(16, 4).text = 'IDR 0' if data['excess_delivery_fee'] =='' else 'IDR '+ str(data['excess_delivery_fee']).replace('.0', '')
        doc.tables[1].cell(16, 7).text = 'IDR 0' if data['excess_delivery_fee'] =='' else 'IDR '+ str(data['excess_delivery_fee']).replace('.0', '')
        doc.tables[1].cell(18, 7).text = 'IDR '+ str(data['total']).replace('.0', '')
        excess =  0 if  data['rx_excess'] == '' else float(data['rx_excess']) + 0 if  data['excess_consult'] == '' else data['excess_consult']
        doc.tables[1].cell(19, 7).text = 'IDR '+ str(data['total'] - excess).replace('.0', '')
        doc.tables[1].cell(20, 7).text = 'IDR '+ str(excess).replace('.0', '')
        
        doc.tables[1].cell(16, 4).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(16, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(18, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(19, 7).paragraphs[0].runs[0].font.size= Pt(8)
        doc.tables[1].cell(20, 7).paragraphs[0].runs[0].font.size= Pt(8)
        
        doc.tables[1].cell(18, 7).paragraphs[0].runs[0].bold = True
        doc.tables[1].cell(19, 7).paragraphs[0].runs[0].bold = True
        doc.tables[1].cell(20, 7).paragraphs[0].runs[0].bold = True
        
        name = data['name'].replace('/', ' ')
        consult_on = data['date'].strftime('%Y-%m-%d')
        consult_id = str(data['consult_id']).replace('.0', '')
        agg = data['aggregator_name']
        payor = data['payor']
        
        if not os.path.exists(f'.tmp/{self.label}/{agg}/{payor}'):
            os.makedirs(f'.tmp/{self.label}/{agg}/{payor}')
        
        
        tmp_name = f'.tmp/{self.label}/{agg}/{payor}/{consult_on}_Consultation_Receipt_{name}_{consult_id}.docx'
        doc.save(tmp_name)
        f.close()
        
        subprocess.run(['libreoffice', '--convert-to', 'pdf' ,
                        tmp_name, '--outdir', f'.tmp/{self.label}/{agg}/{payor}']
                       ,stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL
                    )
        os.remove(tmp_name)
        
    def chunks(self, lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n] 
        
    def chunk_and_zip(self):
        if not os.path.exists(f'output/{self.label}'):
            os.makedirs(f'output/{self.label}')
            
        fls = glob.glob(f'.tmp/{self.label}/*/*/*')
        unique_agg = np.unique([i.split('/')[2] for i in fls])
        unique_pay = np.unique([i.split('/')[3] for i in fls])
        for i in unique_agg:
            for j in unique_pay:
                ij = [x for x in fls if x.split('/')[2] == i and x.split('/')[3] == j]
                k = 1
                for files in self.chunks(ij, 100):
                    with ZipFile(f'output/{self.label}/{i}_{j}_{k}.zip','w') as zip: 
                        for file_each in files:
                            zip.write(file_each, file_each.split('/')[-1]) 
                    k = k + 1
                    
        fls2 = glob.glob(f'output/{label}/*')
        with ZipFile(f'output/{label}.zip','w') as zip:
            for file_each in fls2:
                zip.write(file_each, file_each.split('/')[-1]) 
        shutil.rmtree(f'output/{label}')

    def run(self):
        for index, row in tqdm(self.dataset.iterrows()):
            self.row_to_pdf(row)

        self.chunk_and_zip()

In [3]:

consult = pd.read_excel('Input/04. Report PPU April 2024.xlsx', sheet_name='Discharge Consult')
consult.columns = consult.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')

consult_mrg = pd.read_excel('Input/sample_consult_file_w_prescription_3.xlsx')

consult_mrg.columns = consult_mrg.columns.str.lower().str.strip().str.replace('[!"#$%&\'()*+,-./:;<=>?@[\\]^`{|}~]','', regex = True).str.replace(' ', '_')
consult_mrg = consult_mrg.merge(consult[['consultation_id','dob','gender']])

/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [16]:
consult_mrg['card_no'] = consult_mrg['card_no'].astype(str)
consult_mrg.to_csv('sample_input.csv', index = False)

In [14]:
consult_mrg['card_no'].astype(str)

0        8000150301320947
1        8000150301293037
2        8000150301336844
3        8000150301311268
4        8000150301269417
               ...       
19738    8360535102147461
19739    8360535102147461
19740    8360535102553693
19741    8360535102507292
19742    8360535202492315
Name: card_no, Length: 19743, dtype: object

In [20]:
consult_mrg = consult_mrg.sort_values(['aggregator_name', 'payor'], ascending = False)
payor_stat = consult_mrg['payor'].value_counts().reset_index()
pyr = payor_stat[payor_stat['count'] < 2].iloc[:3].payor

In [7]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [8]:
z = consult_mrg[consult_mrg['payor'].isin(pyr)]
z

NameError: name 'pyr' is not defined

In [53]:
MailMerge(z, 'Week1').run()

3it [00:09,  3.28s/it]


In [47]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n] 
            
def chunk_and_zip(label):
    if not os.path.exists(f'output/{label}'):
        os.makedirs(f'output/{label}')
    else:
        shutil.rmtree(f'output/{label}')

    if os.path.exists(f'output/{label}.zip'):
        os.remove(f'output/{label}.zip')
    
    fls = glob.glob(f'.tmp/{label}/*/*/*')
    unique_agg = np.unique([i.split('/')[2] for i in fls])
    unique_pay = np.unique([i.split('/')[3] for i in fls])
    for i in unique_agg:
        for j in unique_pay:
            ij = [x for x in fls if x.split('/')[2] == i and x.split('/')[3] == j]
            k = 1
            for files in self.chunks(ij, 100):
                with ZipFile(f'output/{label}/{i}_{j}_{k}.zip','w') as zip: 
                    for file_each in files:
                        zip.write(file_each, file_each.split('/')[-1]) 
                k = k + 1
    fls2 = glob.glob(f'output/{label}/*')
    with ZipFile(f'output/{label}.zip','w') as zip:
        for file_each in fls2:
            zip.write(file_each)
    shutil.rmtree('output/{label}')

In [38]:
label = 'Week1'

In [42]:
    fls = glob.glob(f'.tmp/{label}/*/*/*')
    unique_agg = np.unique([i.split('/')[2] for i in fls])
    unique_pay = np.unique([i.split('/')[3] for i in fls])
 

In [43]:
fls

['.tmp/Week1/Admedika/HUSKY/2024-04-10_Consultation_Receipt_FARIS SYAUKI_1381139.pdf',
 '.tmp/Week1/Admedika/ESMAL/2024-04-24_Consultation_Receipt_RAMDHAN SUPRAYOGI_1397955.pdf',
 '.tmp/Week1/Admedika/ASABRI/2024-04-09_Consultation_Receipt_ANDRY MULYAWAN_1380230.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-28_Consultation_Receipt_Andrie Batista Azizul Hakim_1401183.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Nalam Amala Syauqia_1402406.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Ahmad Raihan Fadhil_1402413.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Desi Kurniawati_1401944.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Muhammad Dhio Haryanto_1402036.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-28_Consultation_Receipt_Fitri Yuliyanti_1403400.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Meilin Bong_1401831.pdf',
 '.tmp/Week1/Admedika/PANFIC/2024-04-27_Consultation_Receipt_Nayla Rama

In [1]:
import os
os.getcwd()

'/home/jovyan/work'

In [32]:
shutil.rmtree('output')
os.mkdir('output')